In [1]:
from openai import OpenAI
import yaml
import os
import numpy as np
import pandas as pd

In [2]:
from gpt.curriculum_api_chain import CurriculumAPI
# from train_ant import Curriculum_Module

api = CurriculumAPI("AntMaze_UMaze", "./gpt/prompt/", "./logs/AntMaze_UMaze_0/")
# module = Curriculum_Module("AntMaze_UMaze", "./environments/Curriculum/envs/AntMaze_UMaze.py", "./logs/AntMaze_UMaze_0/")

In [3]:
task_details = api.generate_curriculum()
print(task_details)

Task 1
Name: Basic Locomotion
Description: Maximize torso_velocity
Reason: The agent needs to first learn how to generate forward movement. Focusing on maximizing torso_velocity ensures the agent learns to coordinate its legs for movement, a foundational skill before navigating to specific targets.

Task 2
Name: Orientation Control
Description: Minimize torso_angular_velocity
Reason: After learning to move, the agent should learn how to maintain a stable orientation. By minimizing torso_angular_velocity, the agent practices maintaining a stable orientation while moving, crucial for efficiently navigating towards goals without unnecessary rotations.

Task 3
Name: Directed Movement
Description: Minimize goal_distance with a denser reward being given as the goal_distance decreases
Reason: This task transitions the agent from basic movement and stability to purposeful navigation. Minimizing goal_distance with a dense reward structure teaches the agent to move towards the goal effectively, 

In [4]:
code = api.generate_rewards(0, [])

"```python
def compute_reward_curriculum(self):
    # Gather necessary observation data
    ant_obs = self.get_ant_obs()
    velocity_weight = 2.0  # Higher weight for current task
    torso_vel = self.torso_velocity(ant_obs)
    forward_velocity = np.linalg.norm(torso_vel)  # We care about the magnitude of velocity for locomotion

    # Reward for maximizing forward movement
    reward_forward = np.tanh(forward_velocity) * velocity_weight

    # Total reward is simply the reward for forward movement in this task as our main focus is on basic locomotion
    reward = reward_forward

    # Reward breakdown for potential debugging and analysis
    reward_dict = {
        'reward_forward': reward_forward,
    }

    return reward, reward_dict
```"
Extracted Code Block:
 def compute_reward_curriculum(self):
    # Gather necessary observation data
    ant_obs = self.get_ant_obs()
    velocity_weight = 2.0  # Higher weight for current task
    torso_vel = self.torso_velocity(ant_obs)
    forw

In [5]:
code_list = [code]
code = api.generate_rewards(1, code_list)
code_list.append(code)

```python
def compute_reward_curriculum(self):
    # Gather necessary observation data
    ant_obs = self.get_ant_obs()
    
    # Weights for each component of the reward function, balancing previous and current tasks
    velocity_weight = 1.0  # Still important but less than current task
    angular_velocity_weight = 2.0  # Higher weight for current task to emphasize learning orientation control
    
    # Compute forward velocity for the basic locomotion task
    torso_vel = self.torso_velocity(ant_obs)
    forward_velocity = np.linalg.norm(torso_vel)  # Magnitude of velocity for locomotion
    
    # Compute angular velocity for the orientation control task
    torso_angular_vel = self.torso_angular_velocity(ant_obs)
    angular_velocity_magnitude = np.linalg.norm(torso_angular_vel)  # Magnitude of angular velocity for orientation control
    
    # Reward components
    reward_forward = np.tanh(forward_velocity) * velocity_weight  # Reward for forward movement from previous task
 

In [6]:
code = api.generate_rewards(2, code_list)
code_list.append(code)

```python
def compute_reward_curriculum(self):
    # Gather necessary observation data
    ant_obs = self.get_ant_obs()
    
    # Weights for each component of the reward function, balancing previous and current tasks
    velocity_weight = 0.5  # Previous task, still relevant but less weight
    angular_velocity_weight = 0.5  # Previous task, orientation control
    goal_distance_weight = 2.0  # Current task, emphasize minimizing distance to goal
    
    # Compute forward velocity for the basic locomotion task
    torso_vel = self.torso_velocity(ant_obs)
    forward_velocity = np.linalg.norm(torso_vel)  # Magnitude of velocity for locomotion
    
    # Compute angular velocity for the orientation control task
    torso_angular_vel = self.torso_angular_velocity(ant_obs)
    angular_velocity_magnitude = np.linalg.norm(torso_angular_vel)  # Magnitude for orientation control
    
    # Compute goal distance for the directed movement task
    goal_distance = self.goal_distance(ant_obs)
  